# Recommender system

can be done in two ways: 
  1. user-based collaborative filtering:
            Build a matrix of things each user bought/viewed/rated
            compute similarity scores between users
            Find users similar to you
            Recommend stuff they bought/viewed/rated that you haven't yet
      Limitations:
            Sparsity : there are more number of people than the number of items
            Scalability: computation increases with increase in number of people.
            interest of people can keep changing.
  
  2. item-based collaborative filtering:
            Recommendations are based on relationship between things instead of people
            there are usually fewer things than people and hence less computation
            things or facts about them don't change
            Harder to game the system ( we can't create fake item unlike humans)
            
            
## How does item-based Collaborative filtering works:
(movie recommendation)

  1. find every pair of movies that are watched by the same person
  2. measure the similarity of their ratings across all users who watched both
  3. sort by movie, then by similarity strength

In [19]:
import pandas as pd
import numpy as np

In [5]:

r_cols = ['user_id', 'movie_id','rating']
ratings = pd.read_csv('./ml-100k/u.data', sep='\t', names=r_cols, usecols=range(3), encoding="ISO-8859-1")

m_cols = ['movie_id', 'title']
movies = pd.read_csv('./ml-100k/u.item', sep='|', names=m_cols, usecols=range(2), encoding="ISO-8859-1")

ratings = pd.merge(movies, ratings)

In [6]:
ratings.head()

movie_id             title  user_id  rating
0         1  Toy Story (1995)      308       4
1         1  Toy Story (1995)      287       5
2         1  Toy Story (1995)      148       4
3         1  Toy Story (1995)      280       4
4         1  Toy Story (1995)       66       3

using **pivot_table_function** which takes a column wise data as input  and groups the entries into a two-dimensional table that provides a multidimensional summarization of the data.

NaN  - not available / null
values - ratings value

In [14]:
movieRatings = ratings.pivot_table(index=['user_id'],columns=['title'],values='rating')
movieRatings.head()

title    'Til There Was You (1997)  1-900 (1994)  101 Dalmatians (1996)  \
user_id                                                                   
0                              NaN           NaN                    NaN   
1                              NaN           NaN                    2.0   
2                              NaN           NaN                    NaN   
3                              NaN           NaN                    NaN   
4                              NaN           NaN                    NaN   

title    12 Angry Men (1957)  187 (1997)  2 Days in the Valley (1996)  \
user_id                                                                 
0                        NaN         NaN                          NaN   
1                        5.0         NaN                          NaN   
2                        NaN         NaN                          NaN   
3                        NaN         2.0                          NaN   
4                        NaN         NaN                          NaN   

title    20,000 Leagues Under the Sea (1954)  2001: A Space Odyssey (1968)  \
user_id                                                                      
0                                        NaN                           NaN   
1                                        3.0                           4.0   
2                                        NaN                           NaN   
3                                        NaN                           NaN   
4                                        NaN                           NaN   

title    3 Ninjas: High Noon At Mega Mountain (1998)  39 Steps, The (1935)  \
user_id                                                                      
0                                                NaN                   NaN   
1                                                NaN                   NaN   
2                                                1.0                   NaN   
3                                                NaN                   NaN   
4                                                NaN                   NaN   

title    ...  Yankee Zulu (1994)  Year of the Horse (1997)  \
user_id  ...                                                 
0        ...                 NaN                       NaN   
1        ...                 NaN                       NaN   
2        ...                 NaN                       NaN   
3        ...                 NaN                       NaN   
4        ...                 NaN                       NaN   

title    You So Crazy (1994)  Young Frankenstein (1974)  Young Guns (1988)  \
user_id                                                                      
0                        NaN                        NaN                NaN   
1                        NaN                        5.0                3.0   
2                        NaN                        NaN                NaN   
3                        NaN                        NaN                NaN   
4                        NaN                        NaN                NaN   

title    Young Guns II (1990)  Young Poisoner's Handbook, The (1995)  \
user_id                                                                
0                         NaN                                    NaN   
1                         NaN                                    NaN   
2                         NaN                                    NaN   
3                         NaN                                    NaN   
4                         NaN                                    NaN   

title    Zeus and Roxanne (1997)  unknown  Á köldum klaka (Cold Fever) (1994)  
user_id                                                                        
0                            NaN      NaN                                 NaN  
1                            NaN      4.0                                 NaN  
2                            NaN      NaN                    

In [16]:
#extracting the users who watched star wars

starWarsRatings = movieRatings['Star Wars (1977)']
starWarsRatings.head()

user_id
0    5.0
1    5.0
2    5.0
3    NaN
4    5.0
Name: Star Wars (1977), dtype: float64

Pandas' corrwith function makes it really easy to compute the pairwise correlation of Star Wars' vector of user rating with every other movie

In [17]:
similarMovies = movieRatings.corrwith(starWarsRatings)
similarMovies = similarMovies.dropna()
df = pd.DataFrame(similarMovies)
df.head(10)

/home/admins/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/home/admins/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


0
title                                        
'Til There Was You (1997)            0.872872
1-900 (1994)                        -0.645497
101 Dalmatians (1996)                0.211132
12 Angry Men (1957)                  0.184289
187 (1997)                           0.027398
2 Days in the Valley (1996)          0.066654
20,000 Leagues Under the Sea (1954)  0.289768
2001: A Space Odyssey (1968)         0.230884
39 Steps, The (1935)                 0.106453
8 1/2 (1963)                        -0.142977

In [18]:
similarMovies.sort_values(ascending=False)

title
Hollow Reed (1996)                        1.0
Man of the Year (1995)                    1.0
Star Wars (1977)                          1.0
Stripes (1981)                            1.0
Full Speed (1996)                         1.0
                                         ... 
Theodore Rex (1995)                      -1.0
I Like It Like That (1994)               -1.0
Two Deaths (1995)                        -1.0
Roseanna's Grave (For Roseanna) (1997)   -1.0
Frankie Starlight (1995)                 -1.0
Length: 1410, dtype: float64

Let's construct a new DataFrame that counts up how many ratings exist for each movie, and also the average rating 

In [20]:
movieStats = ratings.groupby('title').agg({'rating': [np.size, np.mean]})
movieStats.head()

rating          
                            size      mean
title                                     
'Til There Was You (1997)      9  2.333333
1-900 (1994)                   5  2.600000
101 Dalmatians (1996)        109  2.908257
12 Angry Men (1957)          125  4.344000
187 (1997)                    41  3.024390

dropping the movies which has rating less than 100

In [38]:
famousmovies = movieStats['rating']['size']>=100
movieStats[famousmovies].sort_values([('rating', 'mean')], ascending=False)

#to check, if movies having less than 200 ratings were removed how it would affect the process
#famousmovies = movieStats['rating']['size']>=200
#movieStats[famousmovies].sort_values([('rating', 'mean')], ascending=False)

rating          
                                   size      mean
title                                            
Close Shave, A (1995)               112  4.491071
Schindler's List (1993)             298  4.466443
Wrong Trousers, The (1993)          118  4.466102
Casablanca (1942)                   243  4.456790
Shawshank Redemption, The (1994)    283  4.445230
...                                 ...       ...
Spawn (1997)                        143  2.615385
Event Horizon (1997)                127  2.574803
Crash (1996)                        128  2.546875
Jungle2Jungle (1997)                132  2.439394
Cable Guy, The (1996)               106  2.339623

[338 rows x 2 columns]

In [39]:
df = movieStats[famousmovies].join(pd.DataFrame(similarMovies, columns=['similarity']))
df.head()

/home/admins/.local/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


(rating, size)  (rating, mean)  similarity
title                                                                   
101 Dalmatians (1996)                    109        2.908257    0.211132
12 Angry Men (1957)                      125        4.344000    0.184289
2001: A Space Odyssey (1968)             259        3.969112    0.230884
Absolute Power (1997)                    127        3.370079    0.085440
Abyss, The (1989)                        151        3.589404    0.203709

In [41]:
'''similarity columns tells how much the movie of that row is similar to that of stars wars (1977)
   rating,size tells how many humans have rated for that movie
   rating, mean tells the mean value of rating'''

df.sort_values(['similarity'], ascending=False)

(rating, size)  \
title                                                                
Star Wars (1977)                                               584   
Empire Strikes Back, The (1980)                                368   
Return of the Jedi (1983)                                      507   
Raiders of the Lost Ark (1981)                                 420   
Austin Powers: International Man of Mystery (1997)             130   
...                                                            ...   
Edge, The (1997)                                               113   
As Good As It Gets (1997)                                      112   
Crash (1996)                                                   128   
G.I. Jane (1997)                                               175   
First Wives Club, The (1996)                                   160   

                                                    (rating, mean)  similarity  
title                                                                           
Star Wars (1977)                                          4.359589    1.000000  
Empire Strikes Back, The (1980)                           4.206522    0.748353  
Return of the Jedi (1983)                                 4.007890    0.672556  
Raiders of the Lost Ark (1981)                            4.252381    0.536117  
Austin Powers: International Man of Mystery (1997)        3.246154    0.377433  
...                                                            ...         ...  
Edge, The (1997)                                          3.539823   -0.127167  
As Good As It Gets (1997)                                 4.196429   -0.130466  
Crash (1996)                                              2.546875   -0.148507  
G.I. Jane (1997)                                          3.360000   -0.176734  
First Wives Club, The (1996)                              3.018750   -0.194496  

[338 rows x 3 columns]